In [1]:
import torch
from pathlib import Path
import numpy as np
import os
import argparse
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.colors as mcolors
import itertools
import seaborn as sns
from tbparse import SummaryReader
import matplotlib.gridspec as gridspec
from tqdm import tqdm
from bokeh.plotting import figure, output_file, save
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, HoverTool, CustomJS, Select
from bokeh.layouts import row, column
from bokeh.io import output_notebook
from bokeh.palettes import RdYlGn11
from bokeh.transform import linear_cmap

from matsimAI.flowsim_dataset import FlowSimDataset

In [2]:
import os
from pathlib import Path

# Get the current working directory
notebook_path = Path("/Users/cta/Documents/GitHub/MatsimAI/matsimAI/notebooks/plot.ipynb").resolve()
os.chdir(notebook_path.parent)

# Go up two directory levels
parent_dir = notebook_path.parent.parent
print(parent_dir)
# cd to root of the repo
os.chdir("../..")

print(os.getcwd())

/Users/cta/Documents/GitHub/MatsimAI/matsimAI
/Users/cta/Documents/GitHub/MatsimAI


In [3]:
def plot_tensor_flows(dataset, predicted_flows, target_flows, link_idx, title, save_path):
    link_id = dataset.edge_mapping.inverse[link_idx]
    pred_link_flows = predicted_flows[link_idx].detach().cpu().numpy()
    target_link_flows = target_flows[link_idx].detach().cpu().numpy()

    hours = np.arange(predicted_flows.shape[1])
    bar_width = 0.4

    plt.figure(figsize=(12, 6))
    plt.bar(hours, pred_link_flows, width=bar_width, color="blue", label="Predicted flows")
    plt.bar(hours + bar_width, target_link_flows, width=bar_width, color="red", label="Target flows")
    plt.xlabel("Hour")
    plt.ylabel("Flows")
    plt.title(title + "\nLink Id: " + str(link_id))
    plt.xticks(hours + bar_width / 2, np.arange(1, predicted_flows.shape[1] + 1))
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    # plt.savefig(save_path)
    # plt.close()
    plt.show()

In [4]:
results_path = "./cluster_results/results/0428162326_nclusters_10_utahnetwork" 
network_path  = ".//cluster_scenarios/utah_10c/utahnetwork.xml" 
counts_path = "./scenarios/cluster_scenarios/utah_10c/utahcounts.xml" 
matsim_output = "./outputs/output_10c" 

results_path = Path(results_path)
network_path = Path(network_path)
counts_path = Path(counts_path)
matsim_output = Path(matsim_output)
# Load the dataset


In [5]:

save_dir = Path(results_path, "analysis")
save_dir.mkdir(exist_ok=True)

last_iter = sorted([int(s.split(".")[1]) for s in os.listdir(matsim_output / "ITERS")])[-1]
last_iteration_path = Path(matsim_output, "ITERS", f"it.{last_iter}")

dataset = FlowSimDataset(results_path.parent, network_path, counts_path, 10)
target_graph = dataset.target_graph
sensor_idxs = dataset.sensor_idxs

# flows = torch.load(results_path / "best_flows.pt")
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
flows = torch.load(results_path / "best_flows.pt", map_location=device)
link_flows = flows["LinkFlows"].to("cpu")
target_flows = target_graph.edge_attr

diff = torch.sum(torch.abs(target_flows[sensor_idxs, :] - link_flows[sensor_idxs, :]), dim=1)
min_sensor_idx = sensor_idxs[torch.argmin(diff)]
max_sensor_idx = sensor_idxs[torch.argmax(diff)]

# Plot tensor flow comparisons
# plot_tensor_flows(dataset, link_flows, target_flows, min_sensor_idx, "Minimum Error Link from Gradient Output",
#                     save_dir / "min_error_link_gradient.png")
    

/var/folders/9t/9q758f_x5rv_4x8n9ttq8kq40000gn/T/ipykernel_93934/2972168772.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  flows = torch.load(results_path / "best_flow

In [6]:

def create_html_plot(dataset, link_flows, hour_count, title, output_html_path="sensor_edges_graph.html"):
    sensor_idxs = dataset.sensor_idxs
    target_flows = dataset.target_graph.edge_attr
    pos = {i: (dataset.target_graph.pos[i][0].item(), dataset.target_graph.pos[i][1].item()) for i in range(len(dataset.target_graph.pos))}

    G_sensor = nx.Graph()

    for i in range(dataset.target_graph.num_edges):
        u, v = dataset.target_graph.edge_index[0][i].item(), dataset.target_graph.edge_index[1][i].item()
        
        if i in sensor_idxs:
            edge_attrs = {"Absolute Difference": [], "Predicted Flow": [], "Target Flow": []}

            for hour_idx in range(hour_count):
                if isinstance(link_flows, torch.Tensor):
                    target_flow = target_flows[i][hour_idx].item()
                    pred_flow = link_flows[i][hour_idx].item()
                elif isinstance(link_flows, pd.DataFrame):
                    link_id = int(dataset.edge_mapping.inv[i])
                    df_row = link_flows[(link_flows['Link Id'] == link_id) & (link_flows['Hour'] == (hour_idx+1))].iloc[0]
                    target_flow = df_row['Count volumes']
                    pred_flow = df_row['MATSIM volumes']
                else:
                    raise ValueError("link_flows must be either a torch.Tensor or a pd.DataFrame")

                abs_diff = abs(target_flow - pred_flow)

                edge_attrs["Absolute Difference"].append(abs_diff)
                edge_attrs["Predicted Flow"].append(pred_flow)
                edge_attrs["Target Flow"].append(target_flow)

            G_sensor.add_edge(u, v, **edge_attrs)
        else:
            G_sensor.add_edge(u, v)

    # Compute max absolute difference for each hour
    max_abs_diff_per_hour = [0] * hour_count
    for hour_idx in range(hour_count):
        max_abs_diff_per_hour[hour_idx] = max(
            [max([data["Absolute Difference"][hour_idx] for u, v, data in G_sensor.edges(data=True) if "Absolute Difference" in data], default=0)],
            default=1e-6  # avoid zero division
        )

    # Build plot data
    sensor_edges = []
    normal_edges = []
    weight_all_hours, pred_all_hours, target_all_hours = [], [], []

    for (u, v, data) in tqdm(G_sensor.edges(data=True), desc="Processing Sensor Edges"):
        if "Absolute Difference" in data:
            sensor_edges.append((u, v, data))
        else:
            normal_edges.append((u, v))

    # Sensor edges (colored)
    s_x0s, s_y0s, s_x1s, s_y1s = [], [], [], []
    mid_x, mid_y = [], []
    for (u, v, data) in sensor_edges:
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        s_x0s.append(x0)
        s_y0s.append(y0)
        s_x1s.append(x1)
        s_y1s.append(y1)
        mid_x.append((x0 + x1) / 2)
        mid_y.append((y0 + y1) / 2)
        weight_all_hours.append(data["Absolute Difference"])
        pred_all_hours.append(data["Predicted Flow"])
        target_all_hours.append(data["Target Flow"])

    sensors_edge_source = ColumnDataSource(data=dict(
        x0=s_x0s, y0=s_y0s, x1=s_x1s, y1=s_y1s,
        mid_x=mid_x, mid_y=mid_y,
        weight=[w[0] for w in weight_all_hours],
        predicted=[p[0] for p in pred_all_hours],
        target=[t[0] for t in target_all_hours],
        weight_all_hours=weight_all_hours,
        predicted_all_hours=pred_all_hours,
        target_all_hours=target_all_hours,
    ))

    # Normal edges (black)
    n_x0s, n_y0s, n_x1s, n_y1s = [], [], [], []
    for (u, v) in normal_edges:
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        n_x0s.append(x0)
        n_y0s.append(y0)
        n_x1s.append(x1)
        n_y1s.append(y1)

    normal_edge_source = ColumnDataSource(data=dict(
        x0=n_x0s, y0=n_y0s, x1=n_x1s, y1=n_y1s,
    ))

    # Create Bokeh figure
    plot = figure(title="Absolute Flow Difference", width=800, height=600, tools="pan,wheel_zoom,box_zoom,reset,save")

    # Loop through each hour to create a color mapper per hour
    for hour_idx in range(hour_count):
        # Color mapper for each hour
        # color_mapper = LinearColorMapper(palette=RdYlGn11, low=0, high=max_abs_diff_per_hour[hour_idx])
        color_mapper = LinearColorMapper(palette=RdYlGn11, low=0, high=max(max_abs_diff_per_hour))

        # Draw sensor edges with color
        plot.segment('x0', 'y0', 'x1', 'y1', source=sensors_edge_source,
                     line_width=10, color={'field': 'weight', 'transform': color_mapper})

    # Draw normal edges in black
    plot.segment('x0', 'y0', 'x1', 'y1', source=normal_edge_source,
                 line_width=1, color="black")

    # Draw nodes
    node_x = [pos[i][0] for i in pos]
    node_y = [pos[i][1] for i in pos]
    plot.scatter(node_x, node_y, size=.01, color="black", alpha=0.1)

    # Hover tool
    tooltips = [("Abs Diff", "@weight"), ("Predicted", "@predicted"), ("Target", "@target")]
    hover = HoverTool(tooltips=tooltips)
    plot.add_tools(hover)

    # Colorbar
    color_bar = ColorBar(color_mapper=color_mapper, location=(0, 0))
    plot.add_layout(color_bar, 'right')


    # === Histogram Setup ===
    bin_edges = np.linspace(0, max(max_abs_diff_per_hour), 21)
    bin_centers = [(bin_edges[i] + bin_edges[i+1])/2 for i in range(len(bin_edges)-1)]
    hist_source = ColumnDataSource(data=dict(
        top=[0]*20,
        bin_center=bin_centers
    ))

    hist_plot = figure(title="Visible Edge Flow Difference Histogram", width=300, height=600, y_range=(0, max(max_abs_diff_per_hour)),
                       toolbar_location=None, tools="")
    hist_plot.hbar(y='bin_center', height=(bin_edges[1] - bin_edges[0]) * 0.8, right='top', left=0, source=hist_source,
                   fill_color=linear_cmap('bin_center', RdYlGn11, low=0, high=max(max_abs_diff_per_hour)), line_color="white")
    hist_plot.xaxis.axis_label = "Count"
    hist_plot.yaxis.axis_label = "Abs Diff"
    hist_plot.ygrid.grid_line_color = None

    # Dropdown for hour selection
    hour_selector = Select(title="Select Hour", value="0", options=[str(i) for i in range(hour_count)])

    # === JS Callback ===
    callback = CustomJS(args=dict(
        source=sensors_edge_source,
        hist_source=hist_source,
        x_range=plot.x_range,
        y_range=plot.y_range,
        hour_selector=hour_selector
    ), code="""
        const hour = parseInt(hour_selector.value);
        const x0 = x_range.start;
        const x1 = x_range.end;
        const y0 = y_range.start;
        const y1 = y_range.end;

        const mids_x = source.data['mid_x'];
        const mids_y = source.data['mid_y'];
        const weights_all = source.data['weight_all_hours'];
        const preds_all = source.data['predicted_all_hours'];
        const targets_all = source.data['target_all_hours'];

        const weights = [];
        for (let i = 0; i < mids_x.length; i++) {
            if (mids_x[i] >= x0 && mids_x[i] <= x1 && mids_y[i] >= y0 && mids_y[i] <= y1) {
                weights.push(weights_all[i][hour]);
            }
            source.data['weight'][i] = weights_all[i][hour];
            source.data['predicted'][i] = preds_all[i][hour];
            source.data['target'][i] = targets_all[i][hour];
        }

        const bin_count = 20;
        const bin_min = 0;
        const bin_max = Math.max(...weights_all.flat());
        const bin_size = (bin_max - bin_min) / bin_count;

        const hist = Array(bin_count).fill(0);
        const bin_centers = [];
        for (let i = 0; i < bin_count; i++) {
            bin_centers.push(bin_min + bin_size * (i + 0.5));
        }

        for (let i = 0; i < weights.length; i++) {
            const w = weights[i];
            const bin = Math.floor((w - bin_min) / bin_size);
            if (bin >= 0 && bin < bin_count) {
                hist[bin]++;
            }
        }

        hist_source.data['top'] = hist;
        hist_source.data['bin_center'] = bin_centers;

        source.change.emit();
        hist_source.change.emit();
    """)

    # Link all events to callback
    plot.x_range.js_on_change("start", callback)
    plot.x_range.js_on_change("end", callback)
    plot.y_range.js_on_change("start", callback)
    plot.y_range.js_on_change("end", callback)
    hour_selector.js_on_change('value', callback)

    layout = row(plot, hist_plot, column(hour_selector))
    # save(layout, filename=output_html_path, title=title)
    # use ipython notebook
    output_notebook()
    show(layout)

create_html_plot(dataset, link_flows, 24, "Gradient Results", Path(save_dir, "gradient_results.html"))


Processing Sensor Edges: 100%|██████████| 3669/3669 [00:00<00:00, 1325030.25it/s]


Loading BokehJS ...

In [10]:
from bokeh.models import Range1d, Label, LinearColorMapper
from bokeh.transform import linear_cmap
# from bokeh.palettes import Viridis256  # Or colorcet.fire, RdYlGn, etc.
from bokeh.palettes import RdYlGn11


def ridge_hist_kde_colormapped(weight_all_hours, bins=30, scale=40):
    hour_count = len(weight_all_hours[0])
    max_val = max(max(w) for w in weight_all_hours)
    mapper = LinearColorMapper(palette=RdYlGn11, low=0, high=max_val)

    p = figure(width=900, height=hour_count * 50, x_range=(0, max_val),
               y_range=Range1d(-scale, (hour_count + 1) * scale * 1.2), toolbar_location=None)

    for h in range(hour_count):
        hour_diffs = [w[h] for w in weight_all_hours if h < len(w)]
        if len(hour_diffs) < 2:
            continue

        baseline = h * scale * 1.2

        # Histogram
        counts, bin_edges = np.histogram(hour_diffs, bins=bins, range=(0, max_val))
        bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
        hist_scaled = counts / counts.max() * scale

        for x, y in zip(bin_centers, hist_scaled):
            p.vbar(x=x, top=y + baseline, bottom=baseline,
                    width=(bin_edges[1] - bin_edges[0]),
                    fill_color={'field': 'x', 'transform': mapper},
                    line_color=None,
                    source=ColumnDataSource(data={'x': [x]}))

        # KDE
        kde = gaussian_kde(hour_diffs, bw_method=0.2)
        x_kde = np.linspace(0, max_val, 500)
        y_kde = kde(x_kde) / max(kde(x_kde)) * scale + baseline

        # One-color KDE (optional: use multi-color with segment() if needed)
        p.line(x_kde, y_kde, line_color="black", line_width=2)

        # Label
        label = Label(x=0, y=baseline + scale * 0.1, text=f"Hour {h}",
                      text_font_size="10pt", text_baseline="middle")
        p.add_layout(label)

    # Final styling
    p.outline_line_color = None
    p.background_fill_color = "#efefef"
    p.yaxis.visible = False
    p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = "#dddddd"
    p.axis.minor_tick_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.axis_line_color = None
    p.xaxis.ticker = FixedTicker(ticks=list(range(0, int(max_val) + 1, 500)))
    p.xaxis.formatter = PrintfTickFormatter(format="%d")

    show(p)
ridge_hist_kde_colormapped(weight_all_hours, bins=30, scale=160)

In [ ]:
import numpy as np
from scipy.stats import gaussian_kde
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColorBar, LinearColorMapper
from bokeh.palettes import Viridis256

output_notebook()

def kde_2d_heatmap(weight_all_hours, bandwidth=0.3):
    hour_count = len(weight_all_hours[0])
    data = []

    for h in range(hour_count):
        for w in weight_all_hours:
            if h < len(w):
                data.append((w[h], h))  # (abs_diff, hour)

    data = np.array(data).T  # shape: (2, N)

    kde = gaussian_kde(data, bw_method=bandwidth)

    # Grid definition
    x_min, x_max = 0, np.percentile(data[0], 99)  # cap outliers
    y_min, y_max = 0, hour_count - 1

    x_grid = np.linspace(x_min, x_max, 300)
    y_grid = np.linspace(y_min, y_max, hour_count * 5)
    xx, yy = np.meshgrid(x_grid, y_grid)
    positions = np.vstack([xx.ravel(), yy.ravel()])

    zz = kde(positions).reshape(yy.shape)

    # Normalize
    zz /= zz.max()

    # Plot
    p = figure(x_range=(x_min, x_max), y_range=(y_min, y_max), width=800, height=400,
               title="2D KDE of Flow Differences Over Hours",
               x_axis_label="Absolute Flow Difference", y_axis_label="Hour")

    color_mapper = LinearColorMapper(palette=Viridis256, low=0, high=zz.max())
    p.image(image=[zz], x=x_min, y=y_min, dw=x_max - x_min, dh=hour_count - 1, color_mapper=color_mapper)

    color_bar = ColorBar(color_mapper=color_mapper, location=(0, 0))
    p.add_layout(color_bar, 'right')

    show(p)

kde_2d_heatmap(weight_all_hours, bandwidth=0.3)

Loading BokehJS ...